## Beautiful Demo

Loads training data and shows an example training face

In [ ]:
from matplotlib import pyplot as plt
from read_data import training_validation

from time import time
from random import choice

(training, train_labels), (test, test_labels) = training_validation()

sample_image = choice(training)

plt.imshow(sample_image)
plt.show()



reading csv: 4.3789589405059814


## Testing Naive Bayes 

Naive bayes based on Normal distribution

In [ ]:
from naive_bayes import naive_bayes
from numpy import zeros, arange
labels = zeros(len(training))
classifier = naive_bayes()

(n_of_imagest, widtht, heigtht) = training.shape
into_linear_featurest = training.reshape(n_of_imagest, widtht * heigtht)

classifier.train(into_linear_featurest, labels)

(n_of_imagesv, widthv, heightv) = validation.shape
into_linear_featuresv = validation.reshape(n_of_imagesv, widthv * heightv)

print(classifier.predict(into_linear_featuresv[0]))

## Example of feature selection used

In [ ]:
from features import *
from processing import IntegralImage

fsize = FeatureSize(0.2, 0.2, 0.4, 0.4)
ii = IntegralImage(sample_image)
ii.process()

f = Feature(fsize, A)
fig, ax1 = plt.subplots(1, 1)
f.print_feature(ii, ax1)
print(f.calculate(ii))

f = Feature(fsize, B1)
fig, ax1 = plt.subplots(1, 1)
f.print_feature(ii, ax1)
print(f.calculate(ii))

f = Feature(fsize, B2)
fig, ax1 = plt.subplots(1, 1)
f.print_feature(ii, ax1)
print(f.calculate(ii))

f = Feature(fsize, C1)
fig, ax1 = plt.subplots(1, 1)
f.print_feature(ii, ax1)
print(f.calculate(ii))

f = Feature(fsize, C2)
fig, ax1 = plt.subplots(1, 1)
f.print_feature(ii, ax1)
print(f.calculate(ii))


plt.show()

## Example of Feature Generation

In [ ]:
from features import *
count = 0
for sz in generate_all_sizes():
    count += 1
    print(sz)
print(count)